In [1]:
from mystem import mystem as mstm

In [14]:
# !curl -sL https://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
# !chmod +x ./mystem

In [2]:
import unicodedata
import re

def remove_accents(input_str):
    nfс_form = unicodedata.normalize('NFC', input_str)
    nfс_form = re.sub(r'[^А-Яа-яЁё\s\-]', u'', nfс_form, flags=re.UNICODE)
    return u"".join([c for c in nfс_form if not unicodedata.combining(c)])

In [3]:
# Выделение лемм из выхлопа mystem
def lemmatized_text_list(contextList):
    """
    Gets lemmas from list of tuples (mystem func)
    """
    line = ''
    textList = list()
    for sentence in contextList:
        if len(sentence) == 0:
            continue
        for word in sentence:
            if word[2] in ['UNKNOWN', 'CONJ', 'INTJ', 'PART', 'PR']:
                continue
            lemma = word[1]
            if lemma != '.':
                if len(lemma) > 1 and lemma[-1:] == '?':
                    lemma = lemma[:-1]                     
                line += lemma + ' '
        textList.append(line)
        line = ''
    return textList

In [4]:
def write_to_file(path, filename, sentencesList):
    outputFile = path + filename
    file_obj = open(outputFile, 'w', encoding="utf8")
    file_obj.writelines(sentencesList)
    file_obj.close()

# Обработка НКРЯ (лучшее)

In [5]:
trainList = ['балка', 'вид', 'винт', 'горн', 'губа', 'жаба', 'клетка',
            'крыло', 'купюра', 'курица', 'лавка', 'лайка', 'лев', 'лира',
            'мина', 'мишень', 'обед', 'оклад', 'опушка', 'полис', 'пост', 
            'поток', 'проказа', 'пропасть', 'проспект', 'пытка', 'рысь',
            'среда', 'хвост', 'штамп'
           ]

In [5]:
trainList = ['акция', 'баба', 'байка', 'бум', 'бычок', 'вал', 'газ', 'гвоздика',
             'гипербола', 'град', 'гусеница', 'дождь', 'домино', 'забой', 'икра',
             'кабачок', 'капот', 'карьер', 'кличка', 'ключ', 'кок', 'кольцо',
             'концерт', 'котелок', 'крона', 'круп', 'кулак', 'лейка', 'лук',
             'мандарин', 'ножка', 'опора', 'патрон', 'печать', 'пол', 'полоз',
             'почерк', 'пробка', 'рак', 'рок', 'свет', 'секрет', 'скат', 'слог',
             'стан', 'стопка', 'таз', 'такса', 'тюрьма', 'шах', 'шашка'
            ]

Скачивание xml-файла под каждое слово.

Хотел сделать считывание через поток, но там были с этим проблемы, поэтому так.

In [7]:
import urllib.request
from urllib.parse import quote

for word in trainList:
    testfile = urllib.request.URLopener()
    file_name = 'http://search1.ruscorpora.ru/download-xml.xml?env=alpha&text=lexgramm&mode=main&sort=gr_tagging&lang=ru&nodia=1&parent1=0&level1=0&lex1={}&gramm1=S%2C(nom%7Cgen%7Cgen2%7Cdat%7Cacc%7Cacc2%7Cins%7Cloc%7Cloc2)%2C(sg%7Cpl)%2Cinan&sem-mod1=sem&sem-mod1=sem2&parent2=0&level2=0&min2=1&max2=1&sem-mod2=sem&sem-mod2=sem2&m2=&p=0&dpp=1000&spd=10&spp=1000&out=kwic&dl=xml'.format(quote(word))
    try:
        testfile.retrieve(file_name, "Data//Original//xml//{}.xml".format(word))
    except:
        print('Error', word)

Выделение текста из каждого xml.

Есть одна проблема. Если брать текст из поля, где содержится текст целиком, то он обрежется, потому что нужное слово выделяется отдельным тегом прямо посреди предложения.

In [8]:
import xml.etree.ElementTree

for word in trainList:
    filepath = 'Data//Original//xml//' + word + '.xml'
    e = xml.etree.ElementTree.parse(filepath).getroot()

    contextList = list()
    tagPrefix = '{urn:schemas-microsoft-com:office:spreadsheet}'

    Worksheet = e.find(tagPrefix + 'Worksheet')
    Table = Worksheet.find(tagPrefix + 'Table')

    count = 0
    for child in Table:
        if child.tag == tagPrefix + 'Row':
            text = ''
            cells = child.findall(tagPrefix + 'Cell')
            cell = cells[-1]
            data = cell.find(tagPrefix + 'Data')
            try:
                text += data.text
            except:
                pass
            text += word + ' '
            cell = cells[5]
            data = cell.find(tagPrefix + 'Data')
            try:
                text += data.text
            except:
                pass

            if text != 'Full context' + word + ' Right context':
                contextList.append(text + '\n')

    write_to_file('Data//Original//txt//', word + '.txt', contextList)

Отчистка текстов от лишних символов и обработка mystem.

In [9]:
%%time
mystemDict = dict()

for word in trainList:
    filename = 'Data//Original//txt//' + word + '.txt'
    with open(filename, 'r', encoding="utf8") as f:
        text = remove_accents(f.read())
        contextList = mstm(text)
        mystemDict[word] = lemmatized_text_list(contextList)

Wall time: 2min 15s


In [32]:
len(mystemDict['гвоздика'])

1563

In [33]:
error_list = mystemDict['гвоздика']
error_list[:10]

['апельсин лимон стакан черносмородиновый джем г консервированный протертый яблоко стакан мадера луковица ст ложка уксус ч ложка картофельный крахмал ч ложка готовый горчица гвоздик лавровый лист ',
 'лук нашинковывать промывать холодный вода полагать кастрюля заливать уксус добавлять лавровый лист перец гвоздик тмин поставлять варить ',
 'фазан стакан белый сухой вино г шпик ст ложка сливочный масло свиной жир стакан сметана луковица лавровый лист ягода можжевельник гвоздик перец соль ',
 'сад этот полынь предпочитать открытый место высаживать она желательно далеко гвоздик георгин бобовый ',
 'несколько долго течение год сохранять всхожесть семя агератум бархатцы вербена гвоздик китайский турецкий календула космея ',
 'дополнять композиция хоста белоокаймленный гвоздик травянка камнеломка ',
 'основа составлять серый гранитный камень образовываться терраса уступ соседствовать миниатюрный хвойник многочисленный очиток камнеломка низкорослый колокольчик гвоздик ',
 'в московский время м

In [36]:
for index, sentence in enumerate(error_list):
    error_list[index] = error_list[index].replace('гвоздик', 'гвоздика')
    
error_list[:10]

['апельсин лимон стакан черносмородиновый джем г консервированный протертый яблоко стакан мадера луковица ст ложка уксус ч ложка картофельный крахмал ч ложка готовый горчица гвоздикаа лавровый лист ',
 'лук нашинковывать промывать холодный вода полагать кастрюля заливать уксус добавлять лавровый лист перец гвоздикаа тмин поставлять варить ',
 'фазан стакан белый сухой вино г шпик ст ложка сливочный масло свиной жир стакан сметана луковица лавровый лист ягода можжевельник гвоздикаа перец соль ',
 'сад этот полынь предпочитать открытый место высаживать она желательно далеко гвоздикаа георгин бобовый ',
 'несколько долго течение год сохранять всхожесть семя агератум бархатцы вербена гвоздикаа китайский турецкий календула космея ',
 'дополнять композиция хоста белоокаймленный гвоздикаа травянка камнеломка ',
 'основа составлять серый гранитный камень образовываться терраса уступ соседствовать миниатюрный хвойник многочисленный очиток камнеломка низкорослый колокольчик гвоздикаа ',
 'в моск

In [10]:
contextDict = dict()

for word in mystemDict:
    
    contextList = mystemDict[word]
    int_before = len(contextList)
       
    # Проверка на наличие слова в тексте (mystem мог поделить контекст на несколько предложений)
    contextList = [x + '\n' for x in contextList if x.find(word) != -1]
    int_middle = len(contextList)
    
    # Удаление предложений, где меньше 3 слов
    # Стоит цифра 5, потому что из-за особенности форматирования по краям стоят пустые строки
    contextList = [' ' + x for x in contextList if len(x.split(' ')) > 5]
    int_after = len(contextList)
    
    contextDict[word] = contextList
    print(word, int_before, int_middle, int_after)

балка 2124 2124 2083
вид 2908 2908 2864
винт 1969 1969 1940
горн 1677 1677 1640
губа 3402 3402 3337
жаба 1201 1200 1149
клетка 3054 3054 2996
крыло 2270 2270 2236
купюра 1574 1574 1549
курица 1942 1942 1862
лавка 2192 2192 2162
лайка 430 430 414
лев 2082 2082 2003
лира 1936 1936 1882
мина 2108 1897 1792
мишень 1803 1803 1773
обед 2461 2461 2325
оклад 1679 1679 1631
опушка 2122 2122 2087
полис 714 714 700
пост 2128 2128 2093
поток 2007 2007 1985
проказа 913 913 895
пропасть 1752 1262 1227
проспект 1751 1751 1715
пытка 1916 1916 1856
рысь 2037 2037 1993
среда 1928 1928 1892
хвост 2275 2275 2215
штамп 1602 1602 1554


Подсчет количества предложений и запись.

In [11]:
for word in trainList:
    wordList = contextDict[word]
    print(len(wordList), word)
    write_to_file('Data//НКРЯ//lemma txt//', word + '.txt', wordList)

2083 балка
2864 вид
1940 винт
1640 горн
3337 губа
1149 жаба
2996 клетка
2236 крыло
1549 купюра
1862 курица
2162 лавка
414 лайка
2003 лев
1882 лира
1792 мина
1773 мишень
2325 обед
1631 оклад
2087 опушка
700 полис
2093 пост
1985 поток
895 проказа
1227 пропасть
1715 проспект
1856 пытка
1993 рысь
1892 среда
2215 хвост
1554 штамп


Я считаю, что в данной задаче кроме НКРЯ использовать ничего не нужно, потому что можно сразу получить большой набор предложений с необходимым словом. Код дальше не был использован в итоговой версии.

# Приведение размеченных данных в презентабельный вид

In [5]:
trainList = ['акция', 'баба', 'байка', 'бум', 'бычок', 'вал', 'газ', 'гвоздика',
             'гипербола', 'град', 'гусеница', 'дождь', 'домино', 'забой', 'икра',
             'кабачок', 'капот', 'карьер', 'кличка', 'ключ', 'кок', 'кольцо',
             'концерт', 'котелок', 'крона', 'круп', 'кулак', 'лейка', 'лук',
             'мандарин', 'ножка', 'опора', 'патрон', 'печать', 'пол', 'полоз',
             'почерк', 'пробка', 'рак', 'рок', 'свет', 'секрет', 'скат', 'слог',
             'стан', 'стопка', 'таз', 'такса', 'тюрьма', 'шах', 'шашка'
            ]

dataset = 'bts-rnc'
mode = 'test'

In [6]:
%%time
mystemDict = dict()
originalDict = dict()

for word in trainList:
    filename = 'Input//txt//' + dataset +'(' + mode + ')' + '//' + word + '.txt'
    with open(filename, 'r', encoding="utf8") as f:
        text = remove_accents(f.read())
        
        sentences = text.split('\n')
        try:
            sentences.remove('')
        except:
            pass
        
        originalDict[word] = sentences
        
        contextList = mstm(text)
        
        textLemmatized = lemmatized_text_list(contextList)
        
        if word == 'гвоздика':
            textLemmatized = [w.replace('гвоздик', 'гвоздика') for w in textLemmatized]
            
        if word == 'круп':
            textLemmatized = [w.replace('крупа', 'круп') for w in textLemmatized]
        
        mystemDict[word] = textLemmatized
        
        print(word, len(contextList) - len(sentences))

акция 0
баба 0
байка 0
бум 0
бычок 0
вал 0
газ 0
гвоздика 0
гипербола 0
град 0
гусеница 0
дождь 0
домино 0
забой 0
икра 0
кабачок 0
капот 0
карьер 0
кличка 0
ключ 0
кок 0
кольцо 0
концерт 0
котелок 0
крона 0
круп 0
кулак 0
лейка 0
лук 0
мандарин 0
ножка 0
опора 0
патрон 0
печать 0
пол 0
полоз 0
почерк 0
пробка 0
рак 0
рок 0
свет 0
секрет 0
скат 0
слог 0
стан 0
стопка 0
таз 0
такса 0
тюрьма 0
шах 0
шашка 0
Wall time: 2min 54s


In [7]:
import csv

words_dict = dict()

for word in trainList:
    
    word_mystem_dict = dict()
    
    filename = 'Input//marked txt//' + dataset +'(' + mode + ')' + '//' + word + '.txt'
    
    with open(filename, 'r', encoding='utf-8', newline='') as f:
        reader = csv.reader(f, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            if len(row) > 1:
                if word == 'круп':
                    word_mystem_dict[row[1].lstrip().replace('крупа', 'круп')] = row[0]
                else:
                    word_mystem_dict[row[1].lstrip()] = row[0]
    
    words_dict[word] = word_mystem_dict

In [8]:
words_dict['круп']

{'акондуктор то трамвай относить пальто барахолка там обменять сметана круп помидор д хармс ': '2',
 'алиса исчезать бежать я густой стадо лошадь показывать дымчатый круп ': '1',
 'анализировать данные приводить таблица можно сделать вывод то рассматривать период потребительский товар много все вывозиться область соль динамика основной прослеживаться незначительный колеблемость признак г отмечаться его большой значение ее производство область круп тенденция сокращение объем ': '2',
 'аппликация мочь быть бумага ход можно пускать разный вид ткань соломка разный вид круп горох фасоль макароны т ': '2',
 'брать долго храниться продукт круп соль сыр растительный масло ': '2',
 'вернее он вылезать круп лошадь он этот звук ': '1',
 'взглядывать глубина горло зеркальце он сказать дочь наш дифтерит тяжелый форма этот болезнь называть круп пом ': '0',
 'война опять подводить тот детский круп ': '0',
 'вопиющий пример новый памятник александр невский стоящий кощунственный отношение лавра выходит

In [9]:
mystemDict['гвоздика'][:5]

['апельсин лимон стакан черносмородиновый джем г консервированный протертый яблоко стакан мадера луковица ст ложка уксус ч ложка картофельный крахмал ч ложка готовый горчица гвоздика лавровый лист ',
 'лук нашинковывать промывать холодный вода полагать кастрюля заливать уксус добавлять лавровый лист перец гвоздика тмин поставлять варить ',
 'фазан стакан белый сухой вино г шпик ст ложка сливочный масло свиной жир стакан сметана луковица лавровый лист ягода можжевельник гвоздика перец соль ',
 'сад этот полынь предпочитать открытый место высаживать она желательно далеко гвоздика георгин бобовый ',
 'несколько долго течение год сохранять всхожесть семя агератум бархатцы вербена гвоздика китайский турецкий календула космея ']

In [8]:
words_labels_dict = dict()

for word in trainList:
    
    print(word)
    print('Размечено:', len(words_dict[word]))
    
    word_labels = dict()
    
    for index, value in enumerate(mystemDict[word]):
        if value in words_dict[word]:
            word_labels[index] = words_dict[word][value]
    
    print('Зафиксировано:', len(word_labels))
    print('-----------')
    
    words_labels_dict[word] = word_labels

акция
Размечено: 134
Зафиксировано: 133
-----------
баба
Размечено: 111
Зафиксировано: 113
-----------
байка
Размечено: 79
Зафиксировано: 79
-----------
бум
Размечено: 56
Зафиксировано: 56
-----------
бычок
Размечено: 66
Зафиксировано: 65
-----------
вал
Размечено: 109
Зафиксировано: 109
-----------
газ
Размечено: 61
Зафиксировано: 61
-----------
гвоздика
Размечено: 75
Зафиксировано: 75
-----------
гипербола
Размечено: 38
Зафиксировано: 39
-----------
град
Размечено: 82
Зафиксировано: 82
-----------
гусеница
Размечено: 43
Зафиксировано: 45
-----------
дождь
Размечено: 56
Зафиксировано: 57
-----------
домино
Размечено: 91
Зафиксировано: 92
-----------
забой
Размечено: 44
Зафиксировано: 44
-----------
икра
Размечено: 74
Зафиксировано: 77
-----------
кабачок
Размечено: 60
Зафиксировано: 60
-----------
капот
Размечено: 64
Зафиксировано: 65
-----------
карьер
Размечено: 18
Зафиксировано: 18
-----------
кличка
Размечено: 64
Зафиксировано: 64
-----------
ключ
Размечено: 91
Зафиксировано: 91
-

In [11]:
words_labels_dict['акция']

{0: '29853',
 1: '15738',
 2: '29853',
 60: '29853',
 69: '29853',
 70: '29853',
 77: '29853',
 86: '29853',
 90: '29853',
 92: '29853',
 93: '29853',
 95: '29853',
 98: '29853',
 99: '29853',
 100: '29853',
 101: '29853',
 103: '29853',
 130: '29853',
 131: '29853',
 132: '29853',
 133: '29853',
 134: '29853',
 155: '29853',
 157: '29853',
 181: '29853',
 182: '29853',
 206: '29853',
 216: '29853',
 219: '15738',
 229: '29853',
 233: '28176',
 245: '29853',
 251: '29853',
 271: '29853',
 316: '15738',
 324: '15738',
 325: '15738',
 337: '29853',
 340: '29853',
 359: '29853',
 360: '29853',
 365: '29853',
 375: '29853',
 376: '29853',
 407: '15738',
 415: '29853',
 434: '15738',
 441: '29853',
 442: '29853',
 483: '15738',
 496: '15738',
 499: '15738',
 517: '29853',
 518: '15738',
 545: '15738',
 573: '29853',
 623: '15738',
 626: '28176',
 627: '28176',
 628: '28176',
 654: '28176',
 723: '15738',
 841: '28176',
 866: '15738',
 896: '29853',
 899: '29853',
 903: '28176',
 904: '28176

In [9]:
wordsMarkedLemma = dict()
wordsMarkedOriginal = dict()

for word in trainList:
    
    markedLemma = list()
    markedOriginal = list()
    
    for index in words_labels_dict[word]:
        
        label = words_labels_dict[word][index]
        
        markedLemma.append(label + '\t' + mystemDict[word][index] + '\n')
        markedOriginal.append(label + '\t' + originalDict[word][index].lstrip() + '\n')
    
    wordsMarkedLemma[word] = markedLemma
    wordsMarkedOriginal[word] = markedOriginal

In [11]:
for word in trainList:
    write_to_file('Gold Standart\\marked txt\\', word + '.csv', wordsMarkedOriginal[word])
    write_to_file('Gold Standart\\marked mystem\\', word + '.csv', wordsMarkedLemma[word])

# Change filenames from ru to en

In [3]:
import unidecode
import pandas as pd

In [6]:
dataset = 'bts-rnc'

wordList = ['акция', 'баба', 'байка', 'бум', 'бычок', 'вал', 'газ', 'гвоздика',
             'гипербола', 'град', 'гусеница', 'дождь', 'домино', 'забой', 'икра',
             'кабачок', 'капот', 'карьер', 'кличка', 'ключ', 'кок', 'кольцо',
             'концерт', 'котелок', 'крона', 'круп', 'кулак', 'лейка', 'лук',
             'мандарин', 'ножка', 'опора', 'патрон', 'печать', 'пол', 'полоз',
             'почерк', 'пробка', 'рак', 'рок', 'свет', 'секрет', 'скат', 'слог',
             'стан', 'стопка', 'таз', 'такса', 'тюрьма', 'шах', 'шашка'
            ]

mode = 'test'

In [2]:
dataset = 'bts-rnc'

wordList = ['балка', 'вид', 'винт', 'горн', 'губа', 'жаба', 'клетка',
            'крыло', 'купюра', 'курица', 'лавка', 'лайка', 'лев', 'лира',
            'мина', 'мишень', 'обед', 'оклад', 'опушка', 'полис', 'пост', 
            'поток', 'проказа', 'пропасть', 'проспект', 'пытка', 'рысь',
            'среда', 'хвост', 'штамп'
           ]
mode = 'train'

In [7]:
labeledPath = "Data/Annotated/lemma/{}({})/".format(dataset, mode)
nonlabeledPath = "Data/Non-Annotated/lemma/{}({})/".format(dataset, mode)

In [8]:
for word in wordList:
    print(word)
    labeledFile = labeledPath + word + '.csv'
    nonlabeledFile = nonlabeledPath + word + '.csv'
    labeledDf = pd.read_csv(labeledFile, header=None, sep='\t', engine='python', encoding='utf-8')
    nonlabeledDf = pd.read_csv(nonlabeledFile, header=None, sep='\t', engine='python', encoding='utf-8')
    
    outputLabelName = labeledPath + unidecode.unidecode(word) + '.csv'
    labeledDf.to_csv(outputLabelName, sep='\t', header=False, index=False, encoding='utf-8')
    
    outputNonLabelName = nonlabeledPath + unidecode.unidecode(word) + '.csv'
    nonlabeledDf.to_csv(outputNonLabelName, sep='\t', header=False, index=False, encoding='utf-8')

акция
баба
байка
бум
бычок
вал
газ
гвоздика
гипербола
град
гусеница
дождь
домино
забой
икра
кабачок
капот
карьер
кличка
ключ
кок
кольцо
концерт
котелок
крона
круп
кулак
лейка
лук
мандарин
ножка
опора
патрон
печать
пол
полоз
почерк
пробка
рак
рок
свет
секрет
скат
слог
стан
стопка
таз
такса
тюрьма
шах
шашка


In [ ]:
x = 'юзер'
unidecode.unidecode(x)